## Testing all available models to see which gives the best accuracy, speed and which models work in europe

This Jupyter Notebook systematically evaluates various Google Gemini models across specified Google Cloud Platform (GCP) regions. Its primary goal is to assess model availability and performance specifically for image analysis tasks relevant to this hackathon project.

The notebook iterates through a predefined list of Gemini models (e.g., `gemini-2.0-flash-001`, `gemini-1.5-pro-001`) and GCP regions (`us-central1`, `europe-west1`, etc.). For each image file found in the `inputs/` directory, it attempts an API call using a basic descriptive prompt to the designated model within the selected region.

Key metrics recorded include:
* API call success/failure status.
* Average processing time per image (in milliseconds) for successful calls.
* Any errors encountered during initialization or processing.

The results are presented in two ways:
1.  A **summary table** (using Pandas) for easy comparison of model availability and speed across different regions.
2.  A **bar chart** (using Matplotlib) visualizing the average processing times for available model/region combinations.

Additionally, the raw text outputs generated by successful model analyses are saved to `outputs/model_test_outputs.json`. This allows for manual review and qualitative assessment of the output precision and relevance for the project's specific needs.

This entire process facilitates the selection of the most suitable *accessible* model and region combination based on empirical performance data and manual quality evaluation.


### All the imports

In [8]:
import os
import time
import logging
import json
from typing import List, Dict, Any
import sys
import traceback

# Project Path Setup
project_root = os.path.abspath('.')  # Assumes notebook is in root
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Library Imports
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import vertexai
    from vertexai.generative_models import GenerativeModel, Part, FinishReason, Image as VertexImage
    from google.api_core import exceptions as google_exceptions
    print("Libraries imported successfully.")
except ImportError as e:
    print(f"Error: Missing libraries. Run 'pip install -r requirements.txt'. Details: {e}")
    raise

# Project Modules
try:
    import config
    try:
        import utils
        print("Utils module loaded.")
    except ImportError:
        print("Utils module not found. Continuing without it.")
        utils = None
    print(f"Config loaded. Project ID: {config.GCP_PROJECT_ID}, Region: {config.GCP_REGION}")
except Exception as e:
    print(f"Error loading config/utils: {e}")
    raise

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("google.api_core").setLevel(logging.WARNING)
logging.getLogger("google.auth").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)


Libraries imported successfully.
Utils module loaded.
Config loaded. Project ID: maximal-cider-458611-r5, Region: europe-west1


### Test Execution & Table Output 

In [9]:
MODELS_TO_TEST = [
    "gemini-2.0-flash-001",
    "gemini-2.0-flash-lite-001",
    "gemini-2.5-flash-preview-04-17",
    "gemini-2.5-pro-preview-03-25",
    "gemini-1.5-flash-002",
    "gemini-1.5-pro-002",
    "gemini-1.5-flash-001",
    "gemini-1.5-pro-001",
    "gemini-1.0-pro-vision-001",
]

REGIONS_TO_TEST = [
    "us-central1",
    "europe-west1",
    "europe-west2",
    "europe-west3",
    "europe-west4"
]

IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp"}
TEST_PROMPT = "Describe this image briefly, focusing on any text present."
DETAILED_OUTPUT_FILENAME = "model_test_outputs.json"

# === Helper Functions ===

def get_image_files(input_dir: str) -> List[str]:
    files = []
    if not os.path.isdir(input_dir):
        logging.error(f"Directory not found: {input_dir}")
        return files
    for fname in os.listdir(input_dir):
        if fname.startswith('.'): continue
        fpath = os.path.join(input_dir, fname)
        if os.path.isfile(fpath) and os.path.splitext(fname.lower())[1] in IMAGE_EXTENSIONS:
            files.append(os.path.abspath(fpath))
    return files

# (Include `test_model_in_region()` function here as-is.)

# === Execute Model Tests and Print Summary Table ===

image_files_to_test = get_image_files(config.INPUT_DIR)
if not image_files_to_test:
    raise FileNotFoundError(f"No images found in {config.INPUT_DIR}")

all_test_results_list = []
detailed_outputs_dict = {}
original_region = config.GCP_REGION

for region in REGIONS_TO_TEST:
    for model_id in MODELS_TO_TEST:
        print(f"\n>>> Testing Model: {model_id} in Region: {region} <<<")
        result_data = test_model_in_region(
            project_id=config.GCP_PROJECT_ID,
            region=region,
            model_id=model_id,
            image_paths=image_files_to_test
        )
        all_test_results_list.append({
            "Model": result_data["model"],
            "Region": result_data["region"],
            "Status": result_data["status"],
            "Avg Time (ms)": result_data["avg_time_ms"],
            "Errors": "; ".join(list(set(result_data["errors"]))) if result_data["errors"] else "None"
        })
        if result_data["status"] == "Available":
            key = f"{region}_{model_id}"
            detailed_outputs_dict[key] = result_data["outputs"]
        time.sleep(2)

# Re-initialize to original region
try:
    if original_region:
        vertexai.init(project=config.GCP_PROJECT_ID, location=original_region)
except Exception as e:
    logging.warning(f"Failed to restore region: {e}")

# Save outputs
os.makedirs(config.OUTPUT_DIR, exist_ok=True)
with open(os.path.join(config.OUTPUT_DIR, DETAILED_OUTPUT_FILENAME), 'w', encoding='utf-8') as f:
    json.dump(detailed_outputs_dict, f, indent=4, ensure_ascii=False)

# === Print Summary Table ===
summary_df = pd.DataFrame(all_test_results_list)
summary_df_display = summary_df.copy()
summary_df_display["Avg Time (ms)"] = summary_df_display["Avg Time (ms)"].apply(lambda x: f"{x:.0f}" if pd.notnull(x) else "N/A")

display(summary_df_display)


2025-05-02 20:43:49,284 - INFO - --- Testing: Initializing Vertex AI for maximal-cider-458611-r5 in us-central1 ---
2025-05-02 20:43:49,285 - INFO - Vertex AI Initialized for region us-central1
2025-05-02 20:43:49,285 - INFO - Instantiating model: gemini-2.0-flash-001
2025-05-02 20:43:49,286 - INFO - Model gemini-2.0-flash-001 instantiated.
2025-05-02 20:43:49,286 - INFO - Processing image: 1.jpeg with gemini-2.0-flash-001 in us-central1



>>> Testing Model: gemini-2.0-flash-001 in Region: us-central1 <<<


2025-05-02 20:43:52,236 - ERROR - Error processing 1.jpeg: UNSPECIFIED
2025-05-02 20:43:52,237 - INFO - Processing image: 2.jpeg with gemini-2.0-flash-001 in us-central1
2025-05-02 20:43:54,868 - ERROR - Error processing 2.jpeg: UNSPECIFIED
2025-05-02 20:43:54,869 - INFO - Processing image: 3.jpeg with gemini-2.0-flash-001 in us-central1
2025-05-02 20:43:56,962 - ERROR - Error processing 3.jpeg: UNSPECIFIED
2025-05-02 20:43:56,963 - INFO - Processing image: 4.jpeg with gemini-2.0-flash-001 in us-central1
2025-05-02 20:43:59,207 - ERROR - Error processing 4.jpeg: UNSPECIFIED
2025-05-02 20:43:59,208 - INFO - Processing image: 5.jpeg with gemini-2.0-flash-001 in us-central1
2025-05-02 20:44:02,104 - ERROR - Error processing 5.jpeg: UNSPECIFIED
2025-05-02 20:44:02,104 - INFO - Processing image: test.jpg with gemini-2.0-flash-001 in us-central1
2025-05-02 20:44:04,557 - ERROR - Error processing test.jpg: UNSPECIFIED
2025-05-02 20:44:04,558 - INFO - Processing image: test.png with gemini-2.0


>>> Testing Model: gemini-2.0-flash-lite-001 in Region: us-central1 <<<


2025-05-02 20:44:10,722 - ERROR - Error processing 1.jpeg: UNSPECIFIED
2025-05-02 20:44:10,722 - INFO - Processing image: 2.jpeg with gemini-2.0-flash-lite-001 in us-central1
2025-05-02 20:44:12,827 - ERROR - Error processing 2.jpeg: UNSPECIFIED
2025-05-02 20:44:12,828 - INFO - Processing image: 3.jpeg with gemini-2.0-flash-lite-001 in us-central1
2025-05-02 20:44:14,817 - ERROR - Error processing 3.jpeg: UNSPECIFIED
2025-05-02 20:44:14,817 - INFO - Processing image: 4.jpeg with gemini-2.0-flash-lite-001 in us-central1
2025-05-02 20:44:16,664 - ERROR - Error processing 4.jpeg: UNSPECIFIED
2025-05-02 20:44:16,665 - INFO - Processing image: 5.jpeg with gemini-2.0-flash-lite-001 in us-central1
2025-05-02 20:44:18,735 - ERROR - Error processing 5.jpeg: UNSPECIFIED
2025-05-02 20:44:18,737 - INFO - Processing image: test.jpg with gemini-2.0-flash-lite-001 in us-central1
2025-05-02 20:44:21,342 - ERROR - Error processing test.jpg: UNSPECIFIED
2025-05-02 20:44:21,344 - INFO - Processing image:

...


>>> Testing Model: gemini-1.5-pro-001 in Region: europe-west4 <<<


2025-05-02 20:47:48,416 - ERROR - Model Not Found error during generate_content for 1.jpeg: 404 Publisher Model `projects/maximal-cider-458611-r5/locations/europe-west4/publishers/google/models/gemini-1.5-pro-001` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions
2025-05-02 20:47:48,417 - INFO - Finished processing images for gemini-1.5-pro-001 in europe-west4. Total time: 0.33s
2025-05-02 20:47:50,420 - INFO - --- Testing: Initializing Vertex AI for maximal-cider-458611-r5 in europe-west4 ---
2025-05-02 20:47:50,420 - INFO - Vertex AI Initialized for region europe-west4
2025-05-02 20:47:50,421 - INFO - Instantiating model: gemini-1.0-pro-vision-001
2025-05-02 20:47:50,422 - INFO - Model gemini-1.0-pro-vision-001 instantiated.
2025-05-02 20:47:50,422 - INFO - Processing image: 1.jpeg with gemini-1.0-pro-vision-001 in europe-wes


>>> Testing Model: gemini-1.0-pro-vision-001 in Region: europe-west4 <<<


2025-05-02 20:47:50,664 - ERROR - Model Not Found error during generate_content for 1.jpeg: 404 Publisher Model `projects/maximal-cider-458611-r5/locations/europe-west4/publishers/google/models/gemini-1.0-pro-vision-001` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions
2025-05-02 20:47:50,665 - INFO - Finished processing images for gemini-1.0-pro-vision-001 in europe-west4. Total time: 0.24s


,Model,Region,Status,Avg Time (ms),Errors
0,gemini-2.0-flash-001,us-central1,Processing Errors,N/A,1.jpeg: Processing Error (AttributeError); tes...
1,gemini-2.0-flash-lite-001,us-central1,Processing Errors,N/A,1.jpeg: Processing Error (AttributeError); tes...
2,gemini-2.5-flash-preview-04-17,us-central1,Processing Errors,N/A,1.jpeg: Processing Error (AttributeError); tes...
3,gemini-2.5-pro-preview-03-25,us-central1,Processing Errors,N/A,1.jpeg: Processing Error (AttributeError); tes...
4,gemini-1.5-flash-002,us-central1,Model Not Found,N/A,1.jpeg: Model Not Found Error
5,gemini-1.5-pro-002,us-central1,Model Not Found,N/A,1.jpeg: Model Not Found Error
6,gemini-1.5-flash-001,us-central1,Model Not Found,N/A,1.jpeg: Model Not Found Error
7,gemini-1.5-pro-001,us-central1,Model Not Found,N/A,1.jpeg: Model Not Found Error
8,gemini-1.0-pro-vision-001,us-central1,Model Not Found,N/A,1.jpeg: Model Not Found Error
9,gemini-2.0-flash-001,europe-west1,Processing Errors,N/A,1.jpeg: Processing Error (AttributeError); tes...


### Graph Rendering

In [10]:
available_df = summary_df[
    (summary_df['Status'] == 'Available') & pd.notnull(summary_df['Avg Time (ms)'])
].copy()

if not available_df.empty:
    available_df['Label'] = available_df['Model'] + '\n@ ' + available_df['Region']
    available_df.sort_values('Avg Time (ms)', inplace=True)

    plt.figure(figsize=(15, 8))
    bars = plt.bar(available_df['Label'], available_df['Avg Time (ms)'], color='skyblue')

    plt.xlabel("Model @ Region")
    plt.ylabel("Average Time per Image (ms)")
    plt.title("Gemini Model Speed Comparison")
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.0f} ms', va='bottom', ha='center', fontsize=8)

    plt.show()
else:
    print("No data available for graph.")


No data available for graph.
